In [1]:
import sys
sys.path.append('../../30_data_tools/')
sys.path.append('../process_masks/')

In [2]:
from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import numpy as np
import cv2
import random
import plotly.express as px
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import pickle
import re
import json
import sqlite3
from datetime import datetime
from skimage import transform
from PIL import ImageDraw, ImageFont
from time import time
import math

In [3]:
from helper import load_dotenv
from get_labelstudio_data import get_moires_of_project, get_results_of_project

In [4]:
from mask_functions import get_config as get_mask_config, load_masks, get_whole_mask
from get_sample_scores import get_images_with_scores

In [5]:
dotenv = load_dotenv()
mask_config = get_mask_config()

In [6]:
DEBUG = False

In [7]:
with open('./config_deform.json') as config_file:
    config = json.load(config_file)

In [8]:
con = sqlite3.connect( dotenv['DB_PATH'] )

In [9]:
moire_pages = set([r['img_name'] for r in get_moires_of_project(1)])

In [10]:
images = get_images_with_scores()

In [11]:
images = images.loc[
    images.job.isin(config['jobs_to_ignore']) == False
]

In [12]:
images = images.loc[
    (images.apply(lambda row: f"{row['job']}.vps2400dpi150lpi.{ row['pdf_filename'] }.4c_300.jpg" not in moire_pages, axis=1)) &
    (images.apply(lambda row: f"{row['job']}.vps2400dpi175lpi.{ row['pdf_filename'] }.4c_300.jpg" not in moire_pages, axis=1))
]

In [13]:
df_moires = pd.DataFrame(
    [
        (r['img_name'],r['rectanglelabels'][0])
        for r in get_results_of_project(2)
    ],
    columns=['pdf_filename','label']
)
df_moires.pdf_filename = df_moires.pdf_filename.str.extract('.+?\.(.+)\.soft_light.+')
df_moires.loc[:,'dummy'] = 1

# Funktionen

In [14]:
def get_basic_name( img_row ):
    cnt = 1

    while len(list(dotenv['GENERIC_GENERATED_DATA_DIR'].glob(f"{img_row['job']}.{img_row['pdf_filename']}.*.{ cnt }*"))) > 0:
        cnt += 1
    
    return f"{img_row['job']}.{img_row['pdf_filename']}.$PLACEHOLDER$.{ cnt }"

In [15]:
def get_random_masks( masks, weights, k ):
    out = []

    while len(out) < k and len(out) < len(masks):
        next_item = random.choices(
            masks,
            weights= weights,
            k=1
        )[0]

        if next_item['bbox'] not in [el['bbox'] for el in out]:
            out.append(next_item)

    return out

In [16]:
def log( message ):
    if DEBUG:
        print( message )

# Ausführung

In [17]:
from pattern_creation import get_pattern_style, get_pattern_img_by_style
from apply_pattern import apply_pattern

In [18]:
0 / 0

ZeroDivisionError: division by zero

In [19]:
rows = []
timestamps = []

for i in tqdm(range(50)):
    try:
        img_row = images.sample(n=1, weights=images.score.apply(lambda val: val - images.score.min())).iloc[0] 
        #img_row = images.loc[1822]
        img_path = dotenv['DATA_DIR'] / img_row['job'] / img_row['variant_name'] / img_row['filename']
        mask_path = dotenv['DATA_DIR'] / img_row['job'] / img_row['variant_name'] / img_row['mask_filename']
        img = Image.open(img_path)
        img_dpi = int(re.match('halftone(\d+)dpi', img_row.variant_name).groups()[0])
        times = []
        times.append((time(),'start'))
        log('start')
    
        if config['processing_dpi'] != img_dpi:
            img = img.resize(
                (int(img.size[0] * (config['processing_dpi'] / img_dpi)), int(img.size[1] * (config['processing_dpi'] / img_dpi)))
            )
        rows_of_image = []
        
        masks = load_masks( mask_path )
    
        for m in masks:
            m['area'] = m['mask'].sum()
        
        # zu verarbeitende Masken auswählen
        relevant_masks = get_random_masks(
            masks,
            [m['area'] * m['stability_score'] for m in masks],
            config['masks_per_generated_image']
        )
        
        multiply_out = np.array(img)
        screen_out = np.array(img)
        softlight_out = np.array(img)
    
        times.append((time(),'Vorbereitung abgeschlossen'))
        log(f'{ times[-1][0] - times[0][0] }: Vorbereitung abgeschlossen')
    
        for i in range(len(relevant_masks)):
            m = relevant_masks[i]
    
            if config['processing_dpi'] != img_dpi:
                mask_scale_factor = config['processing_dpi'] / img_dpi
                m['mask'] = (cv2.resize(
                    (m['mask'] * 255).astype('uint8'),
                    (0,0),
                    fx=mask_scale_factor,
                    fy=mask_scale_factor
                ) / 255).round().astype('bool')
                m['bbox'] = [int(val * mask_scale_factor) for val in m['bbox']]
                
            
            # die verwendeten Anpassungen werden ausgewählt
            row = {
                'image' : img_path.name.strip( img_path.suffix ),
                'img_path' : img_path,
                'method' : 'soft_light', #'screen' if random.random() > 0.5 else 'multiply',
                'ssim' : -1,
                'bbox' : m['bbox'],
                'overlay_intensity_C' : config['overlay_intensity'][0],
                'overlay_intensity_M' : config['overlay_intensity'][1],
                'overlay_intensity_Y' : config['overlay_intensity'][2],
                'overlay_intensity_K' : config['overlay_intensity'][3]
            }
    
            pattern_style = get_pattern_style(config)
            for key in pattern_style:
                row[key] = pattern_style[key]
    
            pattern_img = get_pattern_img_by_style(row, config)
    
            center = (
                int(m['bbox'][0] + m['bbox'][2] / 2),
                int(m['bbox'][1] + m['bbox'][3] / 2)
            )
            
            
            times.append((time(), f'Maske geladen'))
            log(f'{ times[-1][0] - times[0][0] }: Maske { i } geladen')
            
            color = [
                0.3,
                0.4,
                0.1,
                0.7
            ]
            ssim_value_C = 1
            ssim_value_M = 1
            ssim_value_Y = 1
            ssim_value_K = 1
    
    
            if config['overlay_intensity'][0] > 0:
                softlight_out[:,:,0],pattern_mask, ssim_value_C = apply_pattern(
                    softlight_out[:,:,0],
                    pattern_img,
                    center,
                    method='soft_light',
                    overlay_weight=config['overlay_weight'] * config['overlay_intensity'][0],
                    mask=m
                )
    
            if config['overlay_intensity'][1] > 0:
                softlight_out[:,:,1],pattern_mask, ssim_value_M = apply_pattern(
                    softlight_out[:,:,1],
                    pattern_img,
                    center,
                    config,
                    method='soft_light',
                    overlay_weight=config['overlay_weight'] * config['overlay_intensity'][1],
                    mask=m,
                    log=log
                )
    
            if config['overlay_intensity'][2] > 0:
                softlight_out[:,:,2],pattern_mask, ssim_value_Y = apply_pattern(
                    softlight_out[:,:,2],
                    pattern_img,
                    center,
                    config,
                    method='soft_light',
                    overlay_weight=config['overlay_weight'] * config['overlay_intensity'][2],
                    mask=m,
                    log=log
                )
    
            if config['overlay_intensity'][3] > 0:
                softlight_out[:,:,3],pattern_mask, ssim_value_K = apply_pattern(
                    softlight_out[:,:,3],
                    pattern_img,
                    center,
                    config,
                    method='soft_light',
                    overlay_weight=config['overlay_weight'] * config['overlay_intensity'][3],
                    mask=m,
                    log=log
                )
    
            row['ssim'] = (ssim_value_C + ssim_value_M + ssim_value_Y + ssim_value_K) / 4
            rows_of_image.append(row)
    
            times.append((time(), f'Maske implementiert'))
            log(f'{ times[-1][0] - times[0][0] }: Maske { i } implementiert')
    
    
        basic_name = get_basic_name( img_row )
    
        if len([r for r in rows_of_image if r['ssim'] > config['max_tile_ssim']]) > 0: 
            Image.fromarray(
                softlight_out, mode="CMYK"
            ).save(
                dotenv['GENERIC_GENERATED_DATA_DIR'] / (basic_name.replace('$PLACEHOLDER$','soft_light') + '.jpg'),
                progressive=True
            )
    
        # rows anhängen
        for r in rows_of_image:
            r['basic_name'] = basic_name
    
        times.append((time(), f'Bild abgeschlossen'))
        log(f'{ times[-1][0] - times[0][0] }: Bild abgeschlossen')
    
        timestamps.append(times)
        rows += rows_of_image        
    except:
        pass

100%|██████████████████████████████████████| 50/50 [1:54:23<00:00, 137.27s/it]


In [ ]:
tms_rows = []

for i in range(len(timestamps)):
    for t in timestamps[i]:
        tms_rows.append({
            'image' : i,
            'timestamp' : t[0] if type(t[0]) == datetime else datetime.fromtimestamp(t[0]),
            'step' : t[1]
        })

tms = pd.DataFrame.from_dict(tms_rows)
tms.loc[
    :,
    'timestamp_diff'
] = tms.timestamp.diff()

In [ ]:
tms.loc[
    :,
    ['step','timestamp_diff']
].groupby('step').mean().sort_values('timestamp_diff',ascending=False)

In [ ]:
tms.groupby(['image','step']).count()

In [20]:
df = pd.DataFrame.from_dict(rows)

df.to_pickle(
    dotenv['GENERIC_INFORMATION_DATA_DIR'] / f'{round(datetime.now().timestamp())}.pkl'
)

In [21]:
for basic_name in tqdm(df.basic_name.unique()):
    img_path = dotenv['GENERIC_GENERATED_DATA_DIR'] / f"{basic_name.replace('$PLACEHOLDER$','soft_light')}.jpg"    

    if img_path.exists():
        my_img = Image.open(img_path).convert('RGB')
        
        rows = df.loc[
            df.basic_name == basic_name,
            ['bbox','ssim']
        ]
        
        for i in range(rows.shape[0]):
            row = rows.iloc[i]
            m = row.bbox
            ssim_value = row.ssim
            color = (255,0,0,255) if ssim_value < config['max_tile_ssim'] else (0,0,255,255)
            font = ImageFont.load_default(size=50)      
            d = ImageDraw.Draw(my_img, 'RGBA')
            d.rectangle(
                [int(m[0]),int(m[1]),int(m[0]+m[2]),int(m[1]+m[3])],
                outline=color,
                width=10
            )
            d.text((int(m[0]),int(m[1])), str(ssim_value), fill=color, font=font)
    
        my_img.save(f'./10_out/{ img_path.name }', progressive=True )

100%|███████████████████████████████████████████| 47/47 [00:39<00:00,  1.20it/s]
